In [1]:
# Given coefficients and values
beta_0 = 50
beta_1 = 20
beta_2 = 0.07
beta_3 = 35
beta_4 = 0.01
beta_5 = -10

# Values for the variables
GPA = 4.0
IQ = 110
Level = 1  # College

# Calculate the salary for a college graduate
salary_college = beta_0 + 35 + beta_1 * GPA + beta_2 * IQ + beta_3 * Level + beta_4 * (GPA * IQ) + beta_5 * (GPA * Level)
salary_college


172.1

In [2]:
# Import necessary libraries
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Load the Carseats dataset
carseats = pd.read_csv('C:/Users/arman/Downloads/ALL+CSV+FILES+-+2nd+Edition+-+corrected/ALL CSV FILES - 2nd Edition/Carseats.csv')


In [3]:
# Convert 'Urban' and 'US' to categorical types
carseats['Urban'] = carseats['Urban'].astype('category')
carseats['US'] = carseats['US'].astype('category')

# Fit a multiple regression model
model_a = ols('Sales ~ Price + Urban + US', data=carseats).fit()
model_a.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     41.52
Date:                Thu, 24 Oct 2024   Prob (F-statistic):           2.39e-23
Time:                        21:04:40   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1863.
Df Residuals:                     396   BIC:                             1879.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       13.0435      0.651     20.036      0.000      11.764      14.323
Urban[T.Yes]    -0.0219      0.272     -0.081      0.936      -0.556       0.512
US[T.Yes]        1.2006      0.259      4.635      0.000       0.691       1.710
Price           -0.0545      0.005    -10.389      0.000      -0.065      -0.044
==============================================================================
Omnibus:                        0.676   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.713   Jarque-Bera (JB):                0.758
Skew:                           0.093   Prob(JB):                        0.684
Kurtosis:                       2.897   Cond. No.                         628.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [4]:
# Fit a smaller model using only the significant predictors Price and US
model_e = ols('Sales ~ Price + US', data=carseats).fit()
model_e.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.239
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     62.43
Date:                Thu, 24 Oct 2024   Prob (F-statistic):           2.66e-24
Time:                        21:04:40   Log-Likelihood:                -927.66
No. Observations:                 400   AIC:                             1861.
Df Residuals:                     397   BIC:                             1873.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     13.0308      0.631     20.652      0.000      11.790      14.271
US[T.Yes]      1.1996      0.258      4.641      0.000       0.692       1.708
Price         -0.0545      0.005    -10.416      0.000      -0.065      -0.044
==============================================================================
Omnibus:                        0.666   Durbin-Watson:                   1.912
Prob(Omnibus):                  0.717   Jarque-Bera (JB):                0.749
Skew:                           0.092   Prob(JB):                        0.688
Kurtosis:                       2.895   Cond. No.                         607.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [5]:
# Get 95% confidence intervals for the coefficients of the smaller model
conf_intervals = model_e.conf_int(alpha=0.05)
conf_intervals


,0,1
Intercept,11.79032,14.271265
US[T.Yes],0.69152,1.707766
Price,-0.06476,-0.044195


In [6]:
# Calculate influence statistics for the smaller model
influence = model_e.get_influence()
summary_frame = influence.summary_frame()

# Identify high leverage points
leverage = summary_frame['hat_diag']
high_leverage_points = carseats[leverage > 2 * (len(model_e.params) / len(carseats))]

# Identify outliers based on standardized residuals
standardized_residuals = summary_frame['standard_resid']
outliers = carseats[abs(standardized_residuals) > 2]

high_leverage_points, outliers


(     Sales  CompPrice  Income  Advertising  Population  Price ShelveLoc  Age  \
 42   10.43         77      69            0          25     24    Medium   50   
 125   9.34         89      78            0         181     49    Medium   43   
 155   7.71         98      72            0          59     69    Medium   65   
 156   7.49        146      34            0         220    157      Good   51   
 159   9.32        119      60            0         372     70       Bad   30   
 165   0.37        147      58            7         100    191       Bad   27   
 171  12.49         93     106           12         416     55    Medium   75   
 174   0.00        139      24            0         358    185    Medium   79   
 191   6.67        156      42           13         170    173      Good   74   
 203   2.05        131      82            0         132    157       Bad   25   
 208   7.78         86      54            0         497     64       Bad   33   
 269   5.01        159      

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

# Load the datasets
train_df = pd.read_csv('C:/Users/arman/Downloads/playground-series-s4e4/train.csv')
test_df = pd.read_csv('C:/Users/arman/Downloads/playground-series-s4e4/test.csv')

# Separate features and target from the training data
X = train_df.drop(columns=['id', 'Rings'])
y = train_df['Rings']
test_ids = test_df['id']
X_test = test_df.drop(columns=['id'])

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['Length', 'Diameter', 'Height', 'Whole weight', 
                                   'Whole weight.1', 'Whole weight.2', 'Shell weight']),
        ('cat', OneHotEncoder(drop='first'), ['Sex'])
    ])

# Define two models
linear_model = Pipeline(steps=[('preprocessor', preprocessor),
                               ('regressor', LinearRegression())])

rf_model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))])

# Split the training data for evaluation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Train and predict with Linear Regression
linear_model.fit(X_train, y_train)
y_pred_linear = linear_model.predict(X_valid)
rmsle_linear = np.sqrt(mean_squared_log_error(y_valid, np.clip(y_pred_linear, a_min=0, a_max=None)))

# Train and predict with Random Forest Regressor
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_valid)
rmsle_rf = np.sqrt(mean_squared_log_error(y_valid, np.clip(y_pred_rf, a_min=0, a_max=None)))

print("RMSLE for Linear Regression:", rmsle_linear)
print("RMSLE for Random Forest Regressor:", rmsle_rf)

# Predict on the test data with the Random Forest model
y_test_pred_rf = rf_model.predict(X_test)

# Prepare the submission file
submission_df = pd.DataFrame({
    'id': test_ids,
    'Rings': np.clip(y_test_pred_rf, a_min=0, a_max=None)  # Ensure no negative predictions
})
submission_df.to_csv('C:/Users/arman/Downloads/playground-series-s4e4/abalone_submission_rf.csv', index=False)

print("Submission file created successfully!")


RMSLE for Linear Regression: 0.16677429076547393
RMSLE for Random Forest Regressor: 0.15600040102121365
Submission file created successfully!
